In [1]:

import os
import sys
import json
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
import vllm
llm = vllm.LLM(
    "/root/models/DeepResearch_Models/DeepResearcher-7b/",
    gpu_memory_utilization=0.95,
    tensor_parallel_size=4,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=15120,
    disable_log_stats=True,
)
from tools_server.util import MessageClient
import yaml
config = yaml.safe_load(open("./tools_server/config.yaml"))
client = MessageClient(config['data_writing_path'],
                       isconsumer = True,
                       oss_access_key_id=config['oss_access_key_id'],
                       oss_access_key_secret=config['oss_access_key_secret'],
                       oss_endpoint=config['oss_endpoint'])


INFO 07-08 13:50:58 [__init__.py:256] Automatically detected platform cuda.
WARNING 07-08 13:50:59 [config.py:2603] Casting torch.bfloat16 to torch.float16.
INFO 07-08 13:51:10 [config.py:583] This model supports multiple tasks: {'score', 'embed', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 07-08 13:51:10 [config.py:1519] Defaulting to use mp for distributed inference
INFO 07-08 13:51:10 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 07-08 13:51:10 [cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
[2025-07-08 13:51:11,648] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-07-08 13:51:11,744 - root - INFO - gcc -pthread -B /root/miniconda3/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /root/miniconda3/include -fPIC -O2 -isystem /root/miniconda3/include -fPIC -c /tmp/tmpix0xfmhc/test.c -o /tmp/tmpix0xfmhc/test.o
2025-07-08 13:51:11,759 - root - INFO - gcc -pthread -B /root/miniconda3/compiler_compat /tmp/tmpix0xfmhc/test.o -laio -o /tmp/tmpix0xfmhc/a.out


WARNING 07-08 13:51:12 [utils.py:2148] CUDA was previously initialized. We must use the `spawn` multiprocessing start method. Setting VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information.
INFO 07-08 13:51:18 [__init__.py:256] Automatically detected platform cuda.
[2025-07-08 13:51:20,584] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO 07-08 13:51:21 [core.py:53] Initializing a V1 LLM engine (v0.8.1) with config: model='/root/models/DeepResearch_Models/DeepResearcher-7b/', speculative_config=None, tokenizer='/root/models/DeepResearch_Models/DeepResearcher-7b/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=15120, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_c

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:03,  1.24s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.60it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.20it/s]


(VllmWorker rank=3 pid=253589) INFO 07-08 13:52:35 [loader.py:429] Loading weights took 3.65 seconds
(VllmWorker rank=2 pid=253318) INFO 07-08 13:52:35 [loader.py:429] Loading weights took 3.78 seconds


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.05it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.09it/s]
(VllmWorker rank=0 pid=253185) 


(VllmWorker rank=1 pid=253247) INFO 07-08 13:52:35 [loader.py:429] Loading weights took 3.69 seconds
(VllmWorker rank=0 pid=253185) INFO 07-08 13:52:35 [loader.py:429] Loading weights took 3.73 seconds
(VllmWorker rank=2 pid=253318) INFO 07-08 13:52:35 [gpu_model_runner.py:1176] Model loading took 3.5551 GB and 3.936653 seconds
(VllmWorker rank=3 pid=253589) INFO 07-08 13:52:35 [gpu_model_runner.py:1176] Model loading took 3.5551 GB and 3.898726 seconds
(VllmWorker rank=1 pid=253247) INFO 07-08 13:52:35 [gpu_model_runner.py:1176] Model loading took 3.5551 GB and 4.124312 seconds
(VllmWorker rank=0 pid=253185) INFO 07-08 13:52:35 [gpu_model_runner.py:1176] Model loading took 3.5551 GB and 4.165492 seconds
INFO 07-08 13:52:40 [kv_cache_utils.py:537] GPU KV cache size: 4,635,680 tokens
INFO 07-08 13:52:40 [kv_cache_utils.py:540] Maximum concurrency for 15,120 tokens per request: 306.59x
INFO 07-08 13:52:40 [kv_cache_utils.py:537] GPU KV cache size: 4,635,680 tokens
INFO 07-08 13:52:40 [kv

In [6]:
system_prompt = r'''## Background information 
* Today is 2025-04-12
* You are Deep AI Research Assistant

The question I give you is a complex question that requires a *deep research* to answer.

I will provide you with two tools to help you answer the question:
* A web search tool to help you perform google search. 
* A webpage browsing tool to help you get new page content.

You don't have to answer the question now, but you should first think about the research plan or what to search next.

Your output format should be one of the following two formats:

<think>
YOUR THINKING PROCESS
</think>
<answer>
YOUR ANSWER AFTER GETTING ENOUGH INFORMATION
</answer>

or

<think>
YOUR THINKING PROCESS
</think>
<tool_call>
YOUR TOOL CALL WITH CORRECT FORMAT
</tool_call>
 hubn8n 8  8 i8ib ii kb8hn
You should always follow the above two formats strictly.
Only output the final answer (in words, numbers or phrase) inside the <answer></answer> tag, without any explanations or extra information. If this is a yes-or-no question, you should only answer yes or no.


# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "web_search", "description": "Search the web for relevant information from google. You should use this tool if the historical page content is not enough to answer the question. Or last search result is not relevant to the question.", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "The query to search, which helps answer the question"}}, "required": ["query"], "minItems": 1, "uniqueItems": true}}}
{"type": "function", "function": {"name": "browse_webpage", "description": "Browse the webpage and return the content that not appeared in the conversation history. You should use this tool if the last action is search and the search result maybe relevant to the question.", "parameters": {"type": "object", "properties": {"url_list": {"type": "array", "description": "The chosen urls from the search result, do not use url that not appeared in the search result. Do not use more than 3 urls."}, "query": {"type": "string", "description": "These queries aim to retrieve information from the URL webpage."}}, "example": {"name": "browse_webpage", "arguments": {"url_list": ["http://www.dada.com", "xxxx"], "query": "xxxx"}}, "uniqueItems": true}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>'''

tokenizer = llm.get_tokenizer()
def parse_response(response_contents, think: bool = True):
    """Parse response to get the thinking process and answer or tool call.
        return: [(is_stop, thinking, answer/tool_call), ...]
    """
    results = []
    for i, content in enumerate(response_contents):
        print(f'{i} content is {content}')
        if think:
            content = "<think>" + content
        if "<think>" in content and "<answer>" in content:
            if "</think>" not in content or "</answer>" not in content:
                results.append((True, "", ""))
            else:
                think = content.split("<think>")[1].split("</think>")[0]
                answer = content.split("<answer>")[1].split("</answer>")[0]
                results.append((True, think, answer))
        elif "<think>" in content and "<tool_call>" in content:
            if "</tool_call>" not in content or "</think>" not in content:
                results.append((True, "", ""))
            else:
                think = content.split("<think>")[1].split("</think>")[0]
                tool_call = content.split("<tool_call>")[1].split("</tool_call>")[0]
                try:
                    tool_call = json.loads(tool_call)
                    results.append((False, think, tool_call))
                except Exception as e:
                    print(f"model tool call format error: {e}")
                    print(i, content)
                    results.append((True, "", ""))
        else:
            results.append((True, "", ""))
    return results


def execute_predictions(
        tool_call_list, total_number
    ) :
    query_contents = [{"idx": tool_call[0], "question": tool_call[1], "think": tool_call[2],
                       "tool_call": tool_call[3], "total_number":total_number} for tool_call in tool_call_list]

    query_contents = client.submit_tasks(query_contents)
    return query_contents
    
def run_question(question):
    i = 0
    messages_list = []
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    messages_list.append(messages)
    activate_list = [0]
    while(i < 10):
        rollings_active = tokenizer.apply_chat_template(messages_list, add_generation_prompt=True, tokenize=False)
        rollings_active = [x + "<think>" for x in rollings_active]
        # print('----input',rollings_active)
        # print(rollings_active[0])
        response = llm.generate(
            rollings_active,
            vllm.SamplingParams(
                temperature=1.0,
                max_tokens=32000,
            ),
            use_tqdm=True,
        )

        
        response = [x.outputs[0].text for x in response]
        results = parse_response(response)

        tool_call_list = []
        activate_list_copy = []   

        for i in range(len(results)):
            if results[i][0]:
                messages_list[activate_list[i]].append({"role": "assistant", "content": "<think>" + response[i]})
            else:
                activate_list_copy.append(activate_list[i])
                tool_call_list.append((activate_list[i], messages_list[activate_list[i]][1]["content"], results[i][1], results[i][2]))
                
        tool_call_list = execute_predictions(tool_call_list,len(messages_list))
        for i in range(len(tool_call_list)):
            messages_list[tool_call_list[i]['idx']].append(
                {
                    "role": "assistant", 
                    "content": "<think>" + tool_call_list[i]['think'] + "</think>", 
                    "tool_calls": [
                                    {
                                        "type": "function", 
                                        "function": tool_call_list[i]['tool_call']
                                    }
                                ]
                }
            )
            messages_list[tool_call_list[i]['idx']].append(
                {
                    "role": "tool", 
                    "name": tool_call_list[i]['tool_call']["name"],
                    "content": tool_call_list[i]['content']
                }
            )
        i += 1
        activate_list = activate_list_copy
        if len(activate_list_copy) == 0:
            break
    return messages_list

    
# run_question("What is the capital of Columbia County?") 
res = run_question("周杰伦最喜欢的NBA球员历史生涯一共获得过多少枚总冠军戒指") 

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it, est. speed input: 585.32 toks/s, output: 64.54 toks/s]


0 content is  
为了回答这个问题，我需要首先找到周杰伦最喜欢哪位NBA球员。之后需要了解这位球员的历史生涯中获得了多少枚总冠军戒指。我将首先进行搜索来获取这些信息。
</think>
<tool_call>
{"name": "web_search", "arguments": {"query": "周杰伦最喜欢的NBA球员"}}
</tool_call>


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.17s/it, est. speed input: 2491.39 toks/s, output: 59.01 toks/s]


0 content is  
从搜索结果中可以看到周杰伦最喜欢的NBA球员是迈克尔乔丹。接下来我需要了解迈克尔乔丹在他的职业生涯中获得了多少枚总冠军戒指。
</think>
<tool_call>
{"name": "web_search", "arguments": {"query": "迈克尔乔丹获得过多少个NBA总冠军"}}
</tool_call>


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it, est. speed input: 4177.21 toks/s, output: 31.90 toks/s]

0 content is  
搜索结果中多次提到迈克尔乔丹获得了6次NBA总冠军。结合这些信息，我可以得出答案。
</think>
<answer>
6
</answer>


In [4]:
# import pandas as pd 

# test = pd.read_parquet("init_sft_train.parquet")
# test.head()

In [5]:
# # 打开输出文件进行写入
# output_file = 'gen_sft.jsonl'
# with open(output_file, "a", encoding="utf-8") as f_out:
#     # 遍历 test['problem'] 列表中的所有问题，跳过已经处理的部分
#     count = 0
#     for i in range(len(test)):
#         q = test['prompts'].values[i]  # 假定 id 列存在
#         response = run_question(q) 
#         new_format_res = {'input':q, 'output': response}  # 包含 id 字段
#         f_out.write(json.dumps(new_format_res, ensure_ascii=False) + "\n")
#         f_out.flush()
#         count += 1
#         print(f"Processed count: {count},  Result: {new_format_res}")